In [2]:
# -*- coding: utf-8 -*-
# RAG (Stuff Documents 체인, 수동 구현) + ConversationBufferMemory + 로컬 파일 + 캐시 임베딩

import os
from dotenv import load_dotenv

# ----- 배운 임포트 우선 -----
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# StuffDocumentsChain (구 버전 경로)
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import LLMChain

# ===== 0) 환경설정 (.env) =====
# .env 파일에 OPENAI_API_KEY 등 중요 키 저장해두셨다고 하셔서 로드합니다.
# 예) OPENAI_API_KEY=sk-xxx
load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY가 .env에 설정되어 있어야 합니다."

# ===== 1) LLM & 캐시 디렉토리 =====
llm = ChatOpenAI(
    temperature=0.1,  # 안정적 응답
)

cache_dir = LocalFileStore("./.cache/")  # 요청사항: 그대로 사용

# ===== 2) 로컬 문서 로딩 & 청킹 =====
# 요청사항: 웹이 아니라 로컬 파일 경로로 로드
loader = UnstructuredFileLoader("/Users/yy/Desktop/study/full-gpt/files/document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

# ===== 3) 임베딩 + 캐시 임베딩 + 벡터스토어 =====
base_embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(base_embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# ===== 4) Stuff Documents 체인 (수동 구현) =====
# 메모리와 함께 사용할 수 있도록 chat_history를 프롬프트에 포함합니다.
stuff_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a careful reading assistant. Use ONLY the provided context to answer. "
            "If the answer is not in the context, say you don't know."
        ),
        # 메모리에서 들어오는 대화 이력을 받을 placeholder
        ("system", "Conversation so far:\n{chat_history}"),
        (
            "human",
            "Question: {question}\n\n"
            "Use these documents as context:\n{context}\n\n"
            "Answer concisely, quote exact phrases when relevant."
        ),
    ]
)

# LLMChain + StuffDocumentsChain 조합으로 "수동" Stuff 체인 구성
llm_chain = LLMChain(llm=llm, prompt=stuff_prompt)
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",  # 프롬프트의 {context}와 연결
)

# ===== 5) ConversationBufferMemory 부여 (수동 연결) =====
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=False,  # 문자열로 합쳐서 프롬프트에 넣고 싶으면 False
)

# ===== 6) RAG 파이프라인 (수동 결합) =====
def retrieve_then_stuff(question: str) -> str:
    """
    1) retriever로 문서 검색
    2) 메모리에서 chat_history를 불러와 prompt 채우기
    3) StuffDocumentsChain 실행
    4) 메모리에 Q/A 저장
    """
    # 검색
    rel_docs = retriever.get_relevant_documents(question)

    # 메모리에서 현재까지의 대화 이력을 문자열로 로드
    mem_vars = memory.load_memory_variables({})
    chat_history = mem_vars.get("chat_history", "")

    # StuffDocumentsChain 실행
    answer = stuff_chain.run(
        {
            "question": question,
            "chat_history": chat_history,
            "input_documents": rel_docs,  # StuffDocumentsChain은 이 키로 문서 받음
        }
    )

    # 메모리 업데이트
    memory.save_context({"question": question}, {"answer": answer})
    return answer

# ===== 7) 과제의 3개 질문 실행 =====
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?",
]

for q in questions:
    a = retrieve_then_stuff(q)
    print(f"\nQ: {q}\nA: {a}\n")


Q: Is Aaronson guilty?
A: Yes, according to the provided context, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with. The protagonist remembers this as he thinks, "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with."


Q: What message did he write in the table?
A: He wrote: "2+2=5"


Q: Who is Julia?
A: Julia is a person who Winston loves and cares for deeply. He cries out for her in a moment of weakness and later wishes for her to be punished instead of him.

